In [1]:
from modules.smpl_model._smpl import SMPL as SMPL_metro
from modules.utils.image_utils import plot_tensor, to_tensor
from modules.smpl_model.config_smpl import *
from modules.smpl_model.smpl_pose2mesh import SMPL
from tqdm import tqdm

In [2]:
import torch
import os.path as osp
import json
import copy
import numpy as np
import os
import pickle as pkl
from PIL import Image
import torch

## SMPL Model Pose2Mesh

In [9]:
smpl = SMPL()
smpl_metro = SMPL_metro()

# Adapt Regressor for 10 PCs
smpl.layer['neutral'].th_shapedirs= smpl.layer['neutral'].th_shapedirs[:,:,:10]
smpl.layer['neutral'].th_betas= smpl.layer['neutral'].th_betas[:,:10]

/Users/clara/Desktop/MeineProjekte/HMR_3DWP/modules/smpl_model/smplpytorch/smplpytorch/pytorch/smpl_layer.py:40: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /Users/distiller/project/conda/conda-bld/pytorch_1623459065530/work/torch/csrc/utils/tensor_numpy.cpp:180.)
  torch.Tensor(smpl_data['betas'].r).unsqueeze(0))


## Prepocess Data Human 3.6M

In [10]:
from modules.utils.data_utils_h36m import get_cam_pose_intr, get_fitting_error
from modules.utils.geometry import get_smpl_coord

### Create datalist of all images with smpl params

In [19]:
#subject_list = [1, 5, 6, 7, 8, 9, 11]
#subject_list = [1, 5, 6, 7, 8]
#subject_list = [1]
#subject_list = [5]
#subject_list = [6]
#subject_list = [7]
#subject_list = [8]
#subject_list = [9]
subject_list = [11]
data_path = '../H36M'
img_dir = osp.join(data_path, 'images')
annot_dir = osp.join(data_path, 'annotations')
fitting_thr = 25  # milimeter --> Threshhold joints from smpl mesh to h36m gt

In [20]:
images = []
cameras = {}
smpl_params = {}
joints = {}
bboxes = {}
#length_sub = {sub: 0 for sub in subject_list}
for subject in subject_list:
    ### Load image annotations
    with open(osp.join(annot_dir, 'Human36M_subject' + str(subject) + '_data.json'), 'r') as f:
        annotations = json.load(f)
    images.extend(annotations['images'])
    ### Load cameras
    with open(osp.join(annot_dir, 'Human36M_subject' + str(subject) + '_camera.json'), 'r') as f:
        cams = json.load(f)
    cameras[str(subject)] = {cam_id: get_cam_pose_intr(cam) for cam_id, cam in cams.items()}
    ### Load fitted smpl parameter
    with open(osp.join(annot_dir, 'Human36M_subject' + str(subject) + '_smpl_param.json'), 'r') as f:
        smpl_params[str(subject)] = json.load(f)
    ### Load 3d Joint ground truth (17x3)
    with open(osp.join(annot_dir, 'Human36M_subject' + str(subject) + '_joint_3d.json'), 'r') as f:
        joints[str(subject)] = json.load(f)   

with open(osp.join(annot_dir, 'h36m_bboxes.json'), 'r') as f:
        bboxes = json.load(f)

In [21]:
id_to_imgs = {} # Maps img-id to img file
id_to_imgs = {img['id']: img for img in images}

datalist = []
num_smpl_param = 0

In [22]:
for img_id, img in tqdm(id_to_imgs.items(), total=len(id_to_imgs.items())):
    img_name =  img['file_name']  
    subject = img['subject']
    action = img['action_idx']
    subaction = img['subaction_idx']
    frame = img['frame_idx']
    
    ### check if smpl parameters exist
    try:
        smpl_param = smpl_params[str(subject)][str(action)][str(subaction)][str(frame)]
    except KeyError:
        continue
    ### check threshhold of h36m gt and smpl-mesh h36m joints
    joint3d_smpl = np.array(smpl_param['fitted_3d_pose'], np.float32)
    joint3d_h36m_gt = np.array(joints[str(subject)][str(action)][str(subaction)][str(frame)],
                        dtype=np.float32)
    if get_fitting_error(joint3d_h36m_gt, joint3d_smpl) > fitting_thr: 
        continue
        
    cam_id = img['cam_idx']
    cam_param = cameras[str(subject)][str(cam_id)]
    cam_pose, cam_intr = torch.FloatTensor(cam_param['cam_pose']), torch.FloatTensor(cam_param['cam_intr'])
    
    beta = torch.FloatTensor(smpl_param['shape'])
    pose = torch.FloatTensor(smpl_param['pose'])
    trans = torch.FloatTensor(smpl_param['trans']) 
    ### World coordinate --> Camera coordinate
    
    bbox = bboxes[str(img_id)]
    datalist.append({
        'img_name': img_name,
        'img_id': img_id,
        'zarr_id': num_smpl_param,
        'betas': beta,
        'poses': pose,
        'trans': trans,
        'bbox': bbox,
        'cam_id': cam_id,
        'subject': subject,
        'cam_pose': cam_pose,
        'cam_intr': cam_intr,
        })
    
    num_smpl_param += 1
datalist = sorted(datalist, key=lambda x: x['img_id'])

100%|██████████| 231151/231151 [00:03<00:00, 65405.81it/s]


In [23]:
len(datalist)

8100

### Create joint datapickle 

In [27]:
#subject_list = [1, 5, 6, 7, 8, 9, 11]
#subject_list = [1, 5, 6, 7, 8]
subject_list = [9, 11]
fitting_thr = 25  # milimeter --> Threshhold joints from smpl mesh to h36m gt

In [28]:
datalist = []
for subj in subject_list:
    print(subj)
    out_dir = '../H36M/data_pickle'
    sub_str = f'{subj}to{subj}'
    data_pickle = osp.join(out_dir, f'datalist_h36m_thr{fitting_thr}_{sub_str}subj.pickle')
    print(data_pickle)
    with open(data_pickle , "rb") as fp:
        datalist_aux = pkl.load(fp)
    datalist = datalist + datalist_aux

9
../H36M/data_pickle/datalist_h36m_thr25_9to9subj.pickle
11
../H36M/data_pickle/datalist_h36m_thr25_11to11subj.pickle


In [32]:
len(datalist)

74032

## Opt: Store as pickle

In [14]:
import copy

In [29]:
out_dir = '../H36M/data_pickle'
sub_str = f'{min(subject_list)}to{max(subject_list)}'

In [30]:
data_path = osp.join(out_dir, f'datalist_h36m_thr{fitting_thr}_{sub_str}subj.pickle')
data_path

'../H36M/data_pickle/datalist_h36m_thr25_9to11subj.pickle'

In [31]:
with open(data_path, 'wb') as fp:
    pkl.dump(datalist, fp)